In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print(train_data['text'])

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 停用詞
stop_words = set(stopwords.words('english'))

def replace_some(Str):
  Str = re.sub(r'http[s]?://\S+', "", Str)
  Str = re.sub(r'@\S+', "", Str)
  Str = re.sub(r"[^a-zA-Z\s]","",Str.lower())
  Str = Str.split()
  Str = [word for word in Str if word not in stop_words]
  return ' '.join(Str) #隔著' '黏合在一起

train_data['text'] = train_data['text'].apply(replace_some)
train_data['keyword'] = train_data['keyword'].fillna("").apply(replace_some)
test_data['text'] = test_data['text'].apply(replace_some)

In [ ]:
max_len = 0
Index = []

def turn_into_sequences(Str):
  global max_len
  global Index
  if Str == "":
      return [0]
  Str = Str.split()
  Str_list = []
  for i in Str:
    if i not in Index:
      Index.append(i)
      Str_list.append(Index.index(i)+1)
    else:
      Str_list.append(Index.index(i)+1)
  if len(Str_list) > max_len:
    max_len = len(Str_list)
  return Str_list

train_data['text'] = train_data['text'].apply(turn_into_sequences)
test_data['text'] = test_data['text'].apply(turn_into_sequences)

In [ ]:
def padding(sequences):
  global max_len
  while max_len>len(sequences):
    sequences.append(0)
  return sequences

train_data['text'] = train_data['text'].apply(padding)
train_data['keyword'] = train_data['keyword'].apply(turn_into_sequences)

In [ ]:
print(train_data['keyword'].head())
print(max_len)

0    [0]
1    [0]
2    [0]
3    [0]
4    [0]
Name: keyword, dtype: object
23


In [ ]:
X = np.array(train_data['text'])
y = np.array(train_data['target'])

print(X)

[list([1, 2, 3, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([8, 9, 10, 11, 12, 13, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([15, 16, 17, 18, 19, 20, 21, 17, 18, 22, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 ...
 list([8096, 8083, 8084, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([174, 3591, 5211, 5197, 101, 766, 5212, 5211, 442, 5213, 811, 5214, 3956, 5215, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([1243, 3875, 12068, 1571, 27, 2429, 6336, 680, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

In [ ]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)  # 文本數據
        self.y = torch.tensor(y, dtype=torch.float)  # 標籤

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 分割訓練和測試集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 創建數據集和數據加載器
train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # 卷積層1，32個3x3的卷積核，ReLU 激活
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # 卷積層2，64個3x3的卷積核，ReLU 激活
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # 卷積層3，128個3x3的卷積核，ReLU 激活
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # 全連接層
        self.fc1 = nn.Linear(23,23)  # 請根據輸入圖像的尺寸調整
        self.fc2 = nn.Linear(23, 10)  # 10 個類別（對應 MNIST 的 10 個數字）

    def forward(self, x):
        # 依次通過各層，並且加上 ReLU 激活函數
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        # 展平操作
        x = x.view(-1, 128 * 3 * 3)

        # 通過全連接層
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x